In [1]:
import jieba
import jieba.analyse
import codecs

In [2]:
jieba.suggest_freq('沙瑞金', True)
jieba.suggest_freq('田国富', True)
jieba.suggest_freq('高育良', True)
jieba.suggest_freq('侯亮平', True)
jieba.suggest_freq('钟小艾', True)
jieba.suggest_freq('陈岩石', True)
jieba.suggest_freq('欧阳菁', True)
jieba.suggest_freq('易学习', True)
jieba.suggest_freq('王大路', True)
jieba.suggest_freq('蔡成功', True)
jieba.suggest_freq('孙连城', True)
jieba.suggest_freq('季昌明', True)
jieba.suggest_freq('丁义珍', True)
jieba.suggest_freq('郑西坡', True)
jieba.suggest_freq('赵东来', True)
jieba.suggest_freq('高小琴', True)
jieba.suggest_freq('赵瑞龙', True)
jieba.suggest_freq('林华华', True)
jieba.suggest_freq('陆亦可', True)
jieba.suggest_freq('刘新建', True)
jieba.suggest_freq('刘庆祝', True)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\69547\AppData\Local\Temp\jieba.cache
Loading model cost 0.625 seconds.
Prefix dict has been built succesfully.


1

In [4]:
with codecs.open('in_the_name_of_people.txt', 'r', encoding='utf-8') as f:
    document = f.read()
    
    document_cut = jieba.cut(document)
    # print ( ' '.join(jieba_cut))  //如果打印结果，则分词效果消失，后面的result无法显示
    result = ' '.join(document_cut)
    #  result = result.encode('utf-8')
    with open('./in_the_name_of_people_segment.txt', 'w', encoding='utf-8') as f2:
        f2.write(result)
        

拿到了分词后的文件，在一般的NLP处理中，会需要去停用词。由于word2vec的算法依赖于上下文，而上下文有可能就是停词。因此对于word2vec，我们可以不用去停词。

　　　　现在我们可以直接读分词后的文件到内存。这里使用了word2vec提供的LineSentence类来读文件，然后套用word2vec的模型。这里只是一个示例，因此省去了调参的步骤，实际使用的时候，你可能需要对我们上面提到一些参数进行调参。

In [6]:
import logging
import os
from gensim.models import word2vec

In [7]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = word2vec.LineSentence('./in_the_name_of_people_segment.txt')
model = word2vec.Word2Vec(sentences, hs=1, min_count=1, window=3, size=100)

2019-03-13 13:10:11,347 : INFO : collecting all words and their counts
2019-03-13 13:10:11,349 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-03-13 13:10:11,397 : INFO : collected 17878 word types from a corpus of 161343 raw words and 2311 sentences
2019-03-13 13:10:11,398 : INFO : Loading a fresh vocabulary
2019-03-13 13:10:11,448 : INFO : effective_min_count=1 retains 17878 unique words (100% of original 17878, drops 0)
2019-03-13 13:10:11,448 : INFO : effective_min_count=1 leaves 161343 word corpus (100% of original 161343, drops 0)
2019-03-13 13:10:11,487 : INFO : deleting the raw counts dictionary of 17878 items
2019-03-13 13:10:11,488 : INFO : sample=0.001 downsamples 38 most-common words
2019-03-13 13:10:11,489 : INFO : downsampling leaves estimated 120578 word corpus (74.7% of prior 161343)
2019-03-13 13:10:11,498 : INFO : constructing a huffman tree from 17878 words
2019-03-13 13:10:11,815 : INFO : built huffman tree with maximum node depth 17



　　　　模型出来了，我们可以用来做什么呢？这里给出三个常用的应用。

　　　　第一个是最常用的，找出某一个词向量最相近的词集合，代码如下：

In [10]:
req_count = 5
for key in model.wv.similar_by_word('沙瑞金', topn =100):
    if len(key[0])==3:
        req_count -= 1
        print(key[0], key[1])
        if req_count == 0:
            break

2019-03-13 13:14:11,763 : INFO : precomputing L2-norms of word weight vectors


高育良 0.9748526215553284
田国富 0.9357808828353882
易学习 0.9340534806251526
侯亮平 0.93131422996521
季昌明 0.9240447878837585


第二个应用是看两个词向量的相近程度，这里给出了书中两组人的相似程度：

In [20]:
print(model.wv.similarity('沙瑞金', '高育良'))
print(model.wv.similarity('李达康', '王大路'))

0.9748527
0.93381447


第三个应用是找出不同类的词，这里给出了人物分类题：

In [22]:
print(model.wv.doesnt_match("沙瑞金 高育良 李达康 刘庆祝".split()))

刘庆祝
